In [1]:
!pip install networkx gensim

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Created wheel for smart-open: filename=smart_open-1.11.1-py3-none-any.whl size=95261 sha256=e74fc6649b5efe41dbe5a996f68262cd77428ba55543a547ab7fedcfc8cf1a23
  Stored in directory: c:\users\yfana\appdata\local\pip\cache\wheels\8a\c2\d3\16192a758e3f7d7661e23d86dc92a65f2e0a8f98600140f24d
Successfully built smart-open


In [1]:
import networkx as nx
import numpy as np
import os
import time
import csv
import pandas as pd
from itertools import chain
from collections import defaultdict
from gensim.models import Word2Vec
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

unable to import 'smart_open.gcs', disabling that module


# Pipeline

### Data Loader

We need to load networks into memory. Usually networks are organized as pairs of nodes. And sometimes different edges have different weights. Hence, we use networkx.DiGraph to store such structure information and attributes.

In [2]:
def load_data(file_name):
    """
    read edges from an edge file
    """
    edges = list()
    df = pd.read_csv(file_name)
    for idx, row in df.iterrows():
        user_id, friends = row["user_id"], eval(row["friends"])
        for friend in friends:
            # add each friend relation as an edge
            edges.append((user_id, friend))
    edges = sorted(edges)
    
    return edges

def generate_false_edges(true_edges, num_false_edges=20732):
    """
    generate false edges given true edges
    """
    nodes = list(set(chain.from_iterable(true_edges)))
    true_edges = set(true_edges)
    false_edges = set()
    
    while len(false_edges) < num_false_edges:
        # randomly sample two different nodes and check whether the pair exisit or not
        head, tail = np.random.choice(nodes, 2)
        if head != tail and (head, tail) not in true_edges and (head, tail) not in false_edges:
            false_edges.add((head, tail))    
    false_edges = sorted(false_edges)
    
    return false_edges

def construct_graph_from_edges(edges):
    """
    generate a directed graph object given true edges
    DiGraph documentation: https://networkx.github.io/documentation/stable/reference/classes/digraph.html
    """
    # convert a list of edges {(u, v)} to a list of edges with weights {(u, v, w)}
    edge_weight = defaultdict(float)
    for e in edges:
        edge_weight[e] += 1.0
    weighed_edge_list = list()
    for e in sorted(edge_weight.keys()):
        weighed_edge_list.append((e[0], e[1], edge_weight[e]))
        
    graph = nx.DiGraph()
    graph.add_weighted_edges_from(weighed_edge_list)
    
    print("number of nodes:", graph.number_of_nodes())
    print("number of edges:", graph.number_of_edges())
    
    return graph

### Random Walk Generator

Random walk generators or random walkers yield random walks that contain both local and higher-order neighborhood information. However, naive non-uniform sampling is very slow, which requires O(n) time complexity. Here alias sampling can reduce the time complexity to O(1) with O(n) space. If you are interested, please see the following blog.

In [3]:
def alias_setup(probs):
    """
    compute utility lists for non-uniform sampling from discrete distributions.
    details: https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    """
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = list()
    larger = list()
    for kk, prob in enumerate(probs):
        q[kk] = K * prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

def get_alias_node(graph, node):
    """
    get the alias node setup lists for a given node.
    """
    # get the unnormalized probabilities with the first-order information
    unnormalized_probs = list()
    for nbr in graph.neighbors(node):
        unnormalized_probs.append(graph[node][nbr]['weight'])
    unnormalized_probs = np.array(unnormalized_probs)
    if len(unnormalized_probs) > 0:
        normalized_probs = unnormalized_probs / unnormalized_probs.sum()
    else:
        normalized_probs = unnormalized_probs
        
    return alias_setup(normalized_probs)
    
def get_alias_edge(graph, src, dst, p=1, q=1):
    """
    get the alias edge setup lists for a given edge.
    """
    # get the unnormalized probabilities with the second-order information
    unnormalized_probs = list()
    for dst_nbr in graph.neighbors(dst):
        if dst_nbr == src: # distance is 0
            unnormalized_probs.append(graph[dst][dst_nbr]['weight'] / p)
        elif graph.has_edge(dst_nbr, src): # distance is 1
            unnormalized_probs.append(graph[dst][dst_nbr]['weight'])
        else: # distance is 2
            unnormalized_probs.append(graph[dst][dst_nbr]['weight'] / q)
    unnormalized_probs = np.array(unnormalized_probs)
    if len(unnormalized_probs) > 0:
        normalized_probs = unnormalized_probs / unnormalized_probs.sum()
    else:
        normalized_probs = unnormalized_probs

    return alias_setup(normalized_probs)

def preprocess_transition_probs(graph, p=1, q=1):
    """
    preprocess transition probabilities for guiding the random walks.
    """
    alias_nodes = dict()
    for node in graph.nodes():
        alias_nodes[node] = get_alias_node(graph, node)

    alias_edges = dict()
    for edge in graph.edges():
        alias_edges[edge] = get_alias_edge(graph, edge[0], edge[1], p=p, q=q)

    return alias_nodes, alias_edges

The difference between DeepWalk and node2vec is how to generate random walks. The former only consider the first-order information while the latter also involves the second-order information.

In [4]:
def alias_draw(J, q):
    """
    draw sample from a non-uniform discrete distribution using alias sampling.
    """
    K = len(J)

    kk = int(np.floor(np.random.rand() * K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]

def generate_first_order_random_walk(graph, alias_nodes, walk_length=10, start_node=None):
    """
    simulate a random walk starting from start node and considering the first order information.
    """
    if start_node == None:
        start_node = np.random.choice(graph.nodes())
    walk = [start_node]
    cur = start_node
    while len(walk) < walk_length:
        cur_nbrs = list(graph.neighbors(cur))
        if len(cur_nbrs) > 0:
            # sample the next node based on alias_nodes
            cur = cur_nbrs[alias_draw(*alias_nodes[cur])]
            walk.append(cur)
        else:
            break

    return walk
    
def generate_second_order_random_walk(graph, alias_nodes, alias_edges, walk_length=10, start_node=None):
    """
    simulate a random walk starting from start node and considering the second order information.
    """
    if start_node == None:
        start_node = np.random.choice(graph.nodes())
    walk = [start_node]
    
    prev = None
    cur = start_node
    while len(walk) < walk_length:
        cur_nbrs = list(graph.neighbors(cur))
        if len(cur_nbrs) > 0:
            if prev is None:
                # sample the next node based on alias_nodes
                prev, cur = cur, cur_nbrs[alias_draw(*alias_nodes[cur])]
            else:
                # sample the next node based on alias_edges
                prev, cur = cur, cur_nbrs[alias_draw(*alias_edges[(prev, cur)])]
            walk.append(cur)
        else:
            break

    return walk

### Network Embedding Algorithms

In [5]:
def build_deepwalk(graph, alias_nodes, node_dim=10, num_walks=10, walk_length=10):
    """
    build a deepwalk model
    """
    print("building a DeepWalk model...", end="\t")
    st = time.time()
    np.random.seed(0)
    nodes = list(graph.nodes())
    walks = list()
    # generate random walks
    for walk_iter in range(num_walks):
        np.random.shuffle(nodes)
        for node in nodes:
            walks.append(generate_first_order_random_walk(
                graph, alias_nodes, walk_length=walk_length, start_node=node))
        
    walk_lens = [len(w) for w in walks]
    if len(walk_lens) > 0:
        avg_walk_len = sum(walk_lens) / len(walk_lens)
    else:
        avg_walk_len = 0.0
    print("number of walks: %d\taverage walk length: %.4f" % (len(walks), avg_walk_len), end="\t")
    
    # train a skip-gram model for these walks
    model = Word2Vec(walks, size=node_dim, window=3, min_count=0, sg=1, workers=os.cpu_count(), iter=10)
    print("trainig time: %.4f" % (time.time()-st))
    
    return model

def build_node2vec(graph, alias_nodes, alias_edges, node_dim=10, num_walks=10, walk_length=10):
    """
    build a node2vec model
    """
    print("building a node2vec model...", end="\t")
    st = time.time()
    np.random.seed(0)
    nodes = list(graph.nodes())
    walks = list()
    # generate random walks
    for walk_iter in range(num_walks):
        np.random.shuffle(nodes)
        for node in nodes:
            walks.append(generate_second_order_random_walk(
                graph, alias_nodes, alias_edges, walk_length=walk_length, start_node=node))
            
    walk_lens = [len(w) for w in walks]
    if len(walk_lens) > 0:
        avg_walk_len = sum(walk_lens) / len(walk_lens)
    else:
        avg_walk_len = 0.0    
    print("number of walks: %d\taverage walk length: %.4f" % (len(walks), avg_walk_len), end="\t")
    
    # train a skip-gram model for these walks
    model = Word2Vec(walks, size=node_dim, window=3, min_count=0, sg=1, workers=os.cpu_count(), iter=10)
    print("trainig time: %.4f" % (time.time()-st))
    
    return model

### Scorer

In [12]:
def get_cosine_sim(model, u, v):
    """
    get the cosine similarity between two nodes
    """
    try:
        u = model.wv.vectors[model.wv.index2word.index(u)]
        v = model.wv.vectors[model.wv.index2word.index(v)]
        return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    except:
        return 0

def get_auc_score(model, true_edges, false_edges):
    """
    get the auc score
    """
    y_true = [1] * len(true_edges) + [0] * len(false_edges)
    
    y_score = list()
    for e in true_edges:
        y_score.append(get_cosine_sim(model, e[0], e[1]))
    for e in false_edges:
        y_score.append(get_cosine_sim(model, e[0], e[1]))
    
    return roc_auc_score(y_true, y_score)

# Link Prediction

Firstly, we need to load edges into memory and use the networkx.DiGraph structure to store the graph.

In [7]:
map_auc_scores = list()
# deepwalk_auc_scores.mean()
# node2vec_auc_scores.mean()
# node_dim
# num_walks
# walk_length
# num_false_edges
# p
# q

In [10]:
user_file = "data/train.csv"
valid_file = "data/valid.csv"
print("In train file:")
train_edges = load_data(user_file)
valid_edges = load_data(valid_file)
graph = construct_graph_from_edges(edges)
print("In valid file:")
valid_graph = construct_graph_from_edges(valid_edges)

In train file:
number of nodes: 8347
number of edges: 100000
In valid file:
number of nodes: 5388
number of edges: 19268


Link prediction is a task to prediction unseen edges based on graph information. Let's use cross validation to check their performance in the link prediction task.

In [17]:
np.random.seed(0)

# parameters need to be tuned
node_dim = 10
num_walks = 10
walk_length = 22
num_false_edges=20732
p = 0.5
q = 0.5

false_edges = generate_false_edges(train_edges+valid_edges,num_false_edges)
    
# construct the graph and preprocess transition probabilities
graph = construct_graph_from_edges(train_edges)
alias_nodes, alias_edges = preprocess_transition_probs(graph, p=p, q=q)
    

# ------------------------------------------ choose one model here--------------------------
model1 = build_deepwalk(graph, alias_nodes,
                        node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
deepwalk_auc_scores = get_auc_score(model1, valid_edges, false_edges)
    
model2 = build_node2vec(graph, alias_nodes, alias_edges,
                        node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
node2vec_auc_scores = get_auc_score(model2, valid_edges, false_edges)


print("DeepWalk: auc score: %.4f" % (deepwalk_auc_scores))
print("node2vec: auc score: %.4f" % (node2vec_auc_scores))

# store the parameters and scores of this trial
vector = (deepwalk_auc_scores,node2vec_auc_scores,node_dim,num_walks,walk_length,num_false_edges,p,q)
map_auc_scores.append(vector)

number of nodes: 8347
number of edges: 100000
building a DeepWalk model...	number of walks: 83470	average walk length: 19.2983	trainig time: 42.8449
building a node2vec model...	number of walks: 83470	average walk length: 19.2451	trainig time: 42.3727
DeepWalk: auc score: 0.9314
node2vec: auc score: 0.9310


In [18]:
# Evaluate valid.csv acccuracy by 'evaluate.py'
with open("data/valid_ans.csv",'w',newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["src","dst","score"])
    for edge in valid_edges:
        writer.writerow([edge[0],edge[1],1])
    for edge in false_edges:
        writer.writerow([edge[0],edge[1],0])

# generate model prediction
with open("data/valid_pred.csv",'w',newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["src","dst","score"])
#------------------------------------------choose model1 or model2 here-----------------------------------------
    for edge in valid_edges:
        writer.writerow([edge[0],edge[1],get_cosine_sim(model1,edge[0],edge[1])])
    for edge in false_edges:
        writer.writerow([edge[0],edge[1],get_cosine_sim(model1,edge[0],edge[1])])

import os
os.system("python evaluate.py")

0

In [32]:
# generate prediction on test data
dataset = pd.read_csv("data/test.csv")
src = dataset['src']
dst = dataset['dst']

with open("data/pre.csv",'w',newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["src","dst","score"])
    for i in range(len(src)):
        writer.writerow([src[i],dst[i],get_cosine_sim(model,src[i],dst[i])])

In [ ]:
# store the tune records
# remember to change the file name
data = {'map_auc_scores':map_auc_scores}
f = open('pkl/1.pkl','wb')
pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
f.close()

### Try those functions

In [63]:
alias_nodes, alias_edges = preprocess_transition_probs(graph, p=2, q=2)
generate_first_order_random_walk(graph, alias_nodes=alias_nodes,
                                 start_node="DBHCFW3mSmmOEpONHVu1rQ", walk_length=10)
generate_second_order_random_walk(graph, alias_nodes=alias_nodes, alias_edges=alias_edges,
                                  start_node="DBHCFW3mSmmOEpONHVu1rQ", walk_length=10)

['DBHCFW3mSmmOEpONHVu1rQ',
 'gW1l1Ubn4CcKjeJ1fR74aw',
 'aomkMAGFrL-gD6cqQjB4bw',
 'GqLTKz0YsSkX-DTc9oHrBg',
 'tOzTi_wIS2Gajy90pscoMw',
 'jbiI4WJrvhE0ia0mEKvz-g',
 '2xVrxhQJUBmOyG4ML77XKw',
 '0sOleKBI26BKfpW0SEG6Fw',
 '0xRAAStEi_dBFLD-1xSMHQ',
 '3uWCWMWcrn8YSH_qBvlr6w']

In [22]:
model = build_deepwalk(graph, alias_nodes, node_dim=10, num_walks=5, walk_length=3)
model = build_node2vec(graph, alias_nodes, alias_edges, node_dim=10, num_walks=5, walk_length=3)
print("node embedding (\"DBHCFW3mSmmOEpONHVu1rQ\"):",
      model.wv.vectors[model.wv.index2word.index("DBHCFW3mSmmOEpONHVu1rQ")])
print("node embedding (\"Fv0e9RIV9jw5TX3ctA1WbA\"):",
      model.wv.vectors[model.wv.index2word.index("Fv0e9RIV9jw5TX3ctA1WbA")])
print("node embedding (\"VWJ8PSz6Sg5_AlBvQyGvpw\"):",
      model.wv.vectors[model.wv.index2word.index("VWJ8PSz6Sg5_AlBvQyGvpw")])
print("true edge (\"DBHCFW3mSmmOEpONHVu1rQ\", \"Fv0e9RIV9jw5TX3ctA1WbA\"):",
      get_cosine_sim(model, "N6ZTMIue-2b30CJv2tyPGg", "N7E-CfqdME28dakWdEKNvw"))
#print("false edge (\"N6ZTMIue-2b30CJv2tyPGg\", \"YwaKGmRNnSa3R3N4Hf9jLw\"):",
#      get_cosine_sim(model, "N6ZTMIue-2b30CJv2tyPGg", "YwaKGmRNnSa3R3N4Hf9jLw"))

node embedding ("DBHCFW3mSmmOEpONHVu1rQ"): [-0.06658216 -1.7147892   0.11804032 -0.49559236  0.5750869   0.49371615
  0.4574471  -0.84304327 -0.8589881  -0.6348172 ]
node embedding ("Fv0e9RIV9jw5TX3ctA1WbA"): [ 0.12236355 -1.8440742  -0.65914124 -1.2017429  -0.13963051  0.04438686
 -0.63091016 -0.16266017 -0.24979222 -0.17289004]
node embedding ("VWJ8PSz6Sg5_AlBvQyGvpw"): [ 0.0982786  -1.8189262  -0.17022008 -0.39542294  0.18176515  0.38275
 -0.07040039 -1.8763589  -0.67781687 -0.5697088 ]
true edge ("DBHCFW3mSmmOEpONHVu1rQ", "Fv0e9RIV9jw5TX3ctA1WbA"): 0.7338195


In [14]:
np.random.seed(0)

# parameters need to be tuned
kfold = 5
node_dim = 10
num_walks = 10
walk_length = 20
num_false_edges=20732
p = 0.5
q = 0.5

# average score from kfold
deepwalk_auc_scores = list()
node2vec_auc_scores = list()

kf = KFold(n_splits=kfold, shuffle=True)
for k, (train_idx, valid_idx) in enumerate(kf.split(edges)):
    print("interation: %d" %(k))
    # generate false_edge
    train_edges = [edges[idx] for idx in train_idx]
    false_edges = generate_false_edges(train_edges+valid_edges,num_false_edges)
    
    # construct the graph and preprocess transition probabilities
    graph = construct_graph_from_edges(train_edges)
    alias_nodes, alias_edges = preprocess_transition_probs(graph, p=p, q=q)
    

# ------------------------------------------ choose one model here--------------------------
    model1 = build_deepwalk(graph, alias_nodes,
                           node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
    deepwalk_auc_scores.append(get_auc_score(model1, valid_edges, false_edges))
    
    model2 = build_node2vec(graph, alias_nodes, alias_edges,
                           node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
    node2vec_auc_scores.append(get_auc_score(model2, valid_edges, false_edges))

deepwalk_auc_scores = np.array(deepwalk_auc_scores)
node2vec_auc_scores = np.array(node2vec_auc_scores)

print("DeepWalk: avg auc score: %.4f\tstd: %.4f" % (deepwalk_auc_scores.mean(), deepwalk_auc_scores.std()))
print("node2vec: avg auc score: %.4f\tstd: %.4f" % (node2vec_auc_scores.mean(), node2vec_auc_scores.std()))
print("target avg auc score: 0.933%")

# store the parameters and scores of this trial
vector = (deepwalk_auc_scores.mean(),node2vec_auc_scores.mean(),node_dim,num_walks,walk_length,num_false_edges,p,q)
map_auc_scores.append(vector)

interation: 0
number of nodes: 8053
number of edges: 80000
building a DeepWalk model...	number of walks: 80530	average walk length: 16.3349	trainig time: 41.1549
building a node2vec model...	number of walks: 80530	average walk length: 16.2764	trainig time: 41.0169
interation: 1
number of nodes: 8080
number of edges: 80000
building a DeepWalk model...	number of walks: 80800	average walk length: 16.0907	trainig time: 38.3973
building a node2vec model...	number of walks: 80800	average walk length: 16.0038	trainig time: 38.9277
interation: 2
number of nodes: 8040
number of edges: 80000
building a DeepWalk model...	number of walks: 80400	average walk length: 16.1958	trainig time: 33.1583
building a node2vec model...	number of walks: 80400	average walk length: 16.1617	trainig time: 35.2580
interation: 3
number of nodes: 8077
number of edges: 80000
building a DeepWalk model...	number of walks: 80770	average walk length: 16.1853	trainig time: 34.2953
building a node2vec model...	number of walk